In [1]:
#importing required libraries
import glob
from imageai.Detection import VideoObjectDetection
import natsort
import numpy as np
import os
import sys
import numpy as np
import cv2
import glob
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, nan, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint

Using TensorFlow backend.

Bad key "text.kerning_factor" on line 4 in
/home/yagnesh/anaconda3/envs/roadcross/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker


def get_direction(videoPath, trackerType, boxes_list, frame_no):

    '''
    Detects direction of each vehicle in the video and returns and a list 
    containing these directions

    Parameters:
    videoPath(string) : path of the video
    trackerType(string) : the openCV tracker type to be used  
    boxes_list(list) : list containing bounding boxes for all detected vehicles 
    frame_no : frame no of the video for which you require directions

    '''
    k = 0 #iterator for frame no
    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    directions = [0]*n
    speed_array = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - 10

    diff_x1 = [0]*n
    diff_x2 = [0]*n

    font = cv2.FONT_HERSHEY_COMPLEX  # fontFace
    fontScale = 1  # fontScale
    fontColor = (255,255,255) # fontColor
    lineType = 2 # lineType

    #reading the video
    cap = cv2.VideoCapture(videoPath)
    cv2.namedWindow(videoPath, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(videoPath, height=760, width=1366)

    success, frame = cap.read()
    if not success:
        print('Failed to read video')
        sys.exit(1)

    center_point_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)/2)

    while cap.isOpened() :
        success, frame = cap.read()
        if not success:
            break
        
        #saving past few frames as we will apply traking in backward direction
        #form the frame_no
        if( k == frame_no - 10):
            frame_b10 = frame
        if ( k == frame_no - 9):
            frame_b9 = frame
        if ( k == frame_no - 8):
            frame_b8 = frame
        if ( k == frame_no - 7):
            frame_b7 = frame
        if ( k == frame_no - 6):
            frame_b6 = frame
        if ( k == frame_no - 5):
            frame_b5 = frame
        if ( k == frame_no - 4):
            frame_b4 = frame
        if ( k == frame_no - 3):
            frame_b3 = frame
        if ( k == frame_no - 2):
            frame_b2 = frame
        if ( k == frame_no - 1):
            frame_b1 = frame

        if k == frame_no:
            #boxes_list = [tuple(l) for l in boxes_list]
            for i in range(n):
                box = boxes_list[i]
                x1[i] = (box[0]+2*box[2])/2
                diff_x1[i] = x1[i] - center_point_x
                multiTracker_back.add(createTrackerByName(trackerType), frame, box)
            
            #all the vehicles from boxes_list added in the traker instance
            #applying vehicle tracking in backward direction
                
            (success_back, boxes_back) = multiTracker_back.update(frame_b1)
            (success_back, boxes_back) = multiTracker_back.update(frame_b2)
            (success_back, boxes_back) = multiTracker_back.update(frame_b3)
            (success_back, boxes_back) = multiTracker_back.update(frame_b4)
            (success_back, boxes_back) = multiTracker_back.update(frame_b5)
            (success_back, boxes_back) = multiTracker_back.update(frame_b6)
            (success_back, boxes_back) = multiTracker_back.update(frame_b7)
            (success_back, boxes_back) = multiTracker_back.update(frame_b8)
            (success_back, boxes_back) = multiTracker_back.update(frame_b9)
            (success_back, boxes_back) = multiTracker_back.update(frame_b10)


            for i in range(n):
                    b = boxes_back[i]
                    x2[i] = (b[0]+2*b[2])/2
                    diff_x2[i] = x2[i] - center_point_x


            for i in range(n):
                x_d =  diff_x2[i] - diff_x1[i]  # distance covered in x direction
                x_distance = x2[i] - x1[i]
                speed_array[i] = x_d / 10  # speed = distance / time
                print(speed_array[i])
                if x_distance > 0:
                    directions[i] = 1
                elif x_distance < 0:
                    directions[i] = -1

            for i in range(n):
                box = boxes_list[i]
                (x,y,w,h) = [int(v) for v in box]
                if(directions[i] == 0): # not moving
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                    cv2.putText(frame, str(speed_array[i]), org=(x,y-20), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType)
                elif(directions[i] == 1): # direction of interest
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
                    cv2.putText(frame, str(speed_array[i]), org=(x,y-20), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType)
                elif(directions[i] == -1): # opposite to direction of interest
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)

            cv2.imshow(videoPath, frame)
            cv2.waitKey(33)
            # break
        k = k+1
        
    cap.release()
    cv2.destroyAllWindows()
    # return np.array(directions)

In [3]:
def get_directions_from_videos(videos_folder, arrays_folder, trackerType = "KCF"):
    arrays = glob.glob(arrays_folder+'/array*.npy')
    arrays = natsort.natsorted(arrays)[101:]
    videos = glob.glob(videos_folder+'/video*.MOV')
    videos = natsort.natsorted(videos)[101:]

    for (fname, vname) in zip(arrays, videos):
        D = [[],[],[],[],[],[],[],[],[],[]] #since we won't be able to have past 10 frames for the first 10 frames
        print("processing ",vname,":")
        bounding_boxes = np.load(fname, allow_pickle=True) #loading the numpy array containing all detected vehicles
        no_frames = bounding_boxes.shape[0]

        for frame_no in range(10, no_frames):

            temp = bounding_boxes[frame_no]
            boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]

            get_direction(vname, trackerType, boxes_list, frame_no)
            # D.append(speed_array)

In [4]:
get_directions_from_videos('/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/', '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays/arrays_v2_RetinaNet/')

processing  /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/video102.MOV :
-0.1
-0.15
-0.5
-0.2
-0.1
-0.1
6.6
-0.15
-0.15
-0.5
-0.15
-0.15
6.7
-0.15
-0.2
-0.45
-0.15
-0.2
-0.05
6.85
-0.1
-0.05
-0.45
-2.95
-0.2
-0.05
7.0
0.0
0.0
-0.35
-3.65
-0.25
7.1
0.05
-4.45
0.05
-4.55
0.0
0.05
0.0
-0.3
0.1
-5.25
-5.25
0.0
0.0
-0.3
0.0
0.05
7.3
-0.05
-0.05
-0.35
-6.1
-0.05
-0.1
-0.35
7.3
0.0
-6.65
-6.55
-0.1
-0.1
-0.4
0.0
-0.1
-0.55
7.3
-0.2
-0.2
-0.5
-6.75
-0.25
-0.1
-0.15
-0.65
7.25
-0.3
-0.3
-0.6
-0.25
-0.75
7.35
-0.2
-0.35
-0.35
-0.7
-0.35
7.25
-0.2
-0.5
-0.45
-0.85
-0.5
-0.45
7.2
-0.4
-0.6
-0.6
-0.75
-0.45
-0.55
-0.5
7.2
-0.5


KeyboardInterrupt: 